In [ ]:
# For development, use local paths.

import sys
sys.path.append("..")

In [ ]:
# Load local
%load_ext autoreload
%autoreload 2

In [174]:
import nonconsumptive as nc
import pandas as pd

# N-grams

N-grams can consist of any number of tokens. Distributing ngrams of any length potentially invites reconstruction, because all ngrams overlap with each other.

There are a couple strategies for avoiding this.

1. Distribute n-grams that are *broken* on certain chracaters. For instance, if you break all 7-grams at the period marker, the worst case scenario becomes that someone can reconstruct each indvidual sentence in a work but not the order of the sentences.

2. Distribute only a subset of bigrams. Stratg is the subject of forthcoming work with Peter Organisciak that will hopefully be incorporated into this package.

In [186]:
import tempfile
from pathlib import Path
import shutil
folder = Path(tempfile.gettempdir() + "/gutenberg")
if folder.exists():
    shutil.rmtree(folder)
folder.mkdir()

gutenberg = nc.Corpus(
    texts = "../sample_inputs/gutenberg/texts",
    metadata = "../sample_inputs/gutenberg/metadata.ndjson",
    dir = folder,
    compression = "gz",
    cache_set = {"tokenization"}
)

In [187]:
from nonconsumptive.corpus import Ngrams, Bigrams, Trigrams

In [188]:
g = Trigrams(gutenberg)
all = []
for trigrams in g:
    all.append(trigrams)
    
import pyarrow as pa
total_trigrams = pa.Table.from_batches(all).to_pandas()



Building tokenization
Building cache


In [277]:
g.arrow_schema.field('count')

pyarrow.Field<count: uint32>

In [192]:
total_trigrams.query("token1.str.match('[A-Z]')").sort_values("count", ascending = True).tail(20)

,token1,token2,token3,count
1349117,I,don,’,153
3299070,Madame,de,Bellegarde,162
1406194,Mrs,.,Hudson,162
3551344,Mr,.,Brand,164
519392,Mr,.,Weston,167
3081643,Mrs,.,Norris,174
1026438,I,',m,176
3535490,Mr,.,Wentworth,184
841000,D,',Arnot,193
3110499,Mr,.,Crawford,199


In [195]:
indexed = total_trigrams.set_index(["token1", "token2"])

In [282]:
chain = ["the", "Rachel"]
print(" ".join(chain), end = " ")
while len(chain) < 30:
    filtered = indexed.loc[chain[-2]].loc[[chain[-1]]]
    sampled = filtered.sample(n=1, weights=filtered.iloc[:,1], )
    new_word = list(filtered['token3'])[0]
    print(new_word, end = " ")
    chain.append(new_word)

the Rachel , weeping for her first love , who in some monomaniac way whatever whales he could be got up ; the man for a spell ; a wondrous 

In [308]:
filtered.sample(n=1, weights=filtered.iloc[:,1])

,token3,count
token2,,
a,picnic,1
